<a href="https://colab.research.google.com/github/cagBRT/timeSeries/blob/main/10d_Multiple_Parallel_Series.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Multiple Parallel Series**

There are times when we may want to predict the next time series step.<br>
This is known as Multiple Parallel Time Series

For example:<br>
Given the following<br>
[5,10,15]<br>
[10,15,20]<br>
[15,20,25]<br>
[20,25,30]<br>
<br><br>
Predict the values for the next step
[25,30,35]

In [ ]:
def split_sequences(sequences, n_steps): 
  X, y = list(), list()
  for i in range(len(sequences)):
    # find the end of this pattern
    end_ix = i + n_steps
    # check if we are beyond the dataset
    if end_ix > len(sequences)-1: break
    # gather input and output parts of the pattern
    seq_x, seq_y = sequences[i:end_ix, :], sequences[end_ix, :] 
    X.append(seq_x)
    y.append(seq_y)
  return array(X), array(y)

In [ ]:
from numpy import array
from numpy import hstack
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers.convolutional import Conv1D
from keras.layers.convolutional import MaxPooling1D
from keras.utils.vis_utils import plot_model

In [ ]:
in_seq1 = array([10, 20, 30, 40, 50, 60, 70, 80, 90])
in_seq2 = array([15, 25, 35, 45, 55, 65, 75, 85, 95])
out_seq = array([in_seq1[i]+in_seq2[i] for i in range(len(in_seq1))])

In [ ]:
in_seq1 = in_seq1.reshape((len(in_seq1), 1)) 
in_seq2 = in_seq2.reshape((len(in_seq2), 1)) 
out_seq = out_seq.reshape((len(out_seq), 1)) 
# horizontally stack columns
dataset = hstack((in_seq1, in_seq2, out_seq))
dataset

In [ ]:
n_steps = 3
n_features = X.shape[2]
# convert into input/output
X, y = split_sequences(dataset, n_steps) 
print(X.shape, y.shape)
# summarize the data
for i in range(len(X)):
  print(X[i], y[i])

In [ ]:
model = Sequential()
model.add(Conv1D(64, 2, activation='relu', input_shape=(n_steps, n_features))) 
model.add(MaxPooling1D())
model.add(Flatten())
model.add(Dense(50, activation='relu'))
model.add(Dense(n_features))

In [ ]:
model.compile(optimizer='adam', loss='mse')
model.fit(X, y, epochs=3000, verbose=0)

In [ ]:
plot_model(model, show_shapes=True, show_layer_names=True)

In [ ]:
x_input = array([[70,75,145], [80,85,165], [90,95,185]]) 
x_input = x_input.reshape((1, n_steps, n_features))
yhat = model.predict(x_input, verbose=0)

In [ ]:
yhat